# Trabalhando com PostgreSQL

### Conexão

- Baixando módulos necessários para conexão

*Foi necessário instalar o psycopg2 via terminal com o seguinte comando ```sudo apt-get install build-dep python-psycopg2``` e depois instalar o binário através do gerenciador **pip** para o módulo ser reconhecido no **env**.*

In [2]:
#%pip install pandas
#%pip install psycopg2-binary
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


- Importando módulos

In [1]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from os import getenv

- Obtendo credenciais de acesso ao PostgreSQL

Foi preciso mudar a variável de ambiente **USER** para **USR** evitando assim o conflito no valor da variável no dotenv e no próprio SO

In [2]:
load_dotenv()

user = getenv('USR') or 'postgres'
passwd = getenv('PASSWD') or 'postgres'
host = getenv('HOST') or 'localhost'
port = getenv('PORT') or '5432'
default_db = getenv('DB') or 'postgres'


- Conectando ao PostgreSQL

In [4]:
def db_connect(host, port, db, user, passwd):
    connect = psycopg2.connect (
        host = host,
        port = port,
        database = db,
        user = user,
        password = passwd
    )

    return connect

db_connect(host, port, default_db, user, passwd)

<connection object at 0x7f7c0bb76980; dsn: 'user=postgres password=xxx dbname=pokedata host=localhost port=5433', closed: 0>

### Carregando dados

In [5]:
df_pk_types = pd.read_csv('./data/pk_types.csv', sep=';')
df_pk_status = pd.read_csv('./data/pk_status.csv', sep=';')
df_pk_gen = pd.read_csv('./data/pk_gen.csv', sep=';')

df_pk_gen.head()

,#,Name,Generation,Legendary
0,1,Bulbasaur,1,False
1,2,Ivysaur,1,False
2,3,Venusaur,1,False
3,3,VenusaurMega Venusaur,1,False
4,4,Charmander,1,False


### Manipulando o banco de dados

- Função para criar tabelas

In [6]:
def create_db(sql):
    connect = db_connect(host, port, default_db, user, passwd)
    cursor = connect.cursor()
    
    cursor.execute(sql)
    connect.commit()
    connect.close()

- Criando a tabela **pk_types** para armazenar:
    - type_id (usado para armazenar o código de identificação de cada tipo)
    - type_name (usado para armazenar o tipo)

In [6]:
pk_types_sql = '''CREATE TABLE IF NOT EXISTS pokedata.public.pk_types (
    type_id                 serial PRIMARY KEY
    ,type_name              VARCHAR(10) NOT NULL
)'''

create_db(pk_types_sql)

 - Criando a tabela **pk_gen** para armazenar:
    - gen_id (usado para identificar a geração e se o pokemon é do tipo lendário ou não)
    - generation (usado para armazenar a geração)
    - legendary (usado para armazenar se é um pokemon lendário ou não)

In [11]:
pk_gen_sql = '''CREATE TABLE IF NOT EXISTS pokedata.public.pk_gen (
    gen_id                  serial PRIMARY KEY
    ,generation             SMALLINT NOT NULL
    ,legendary              BOOL DEFAULT false
)'''

create_db(pk_gen_sql)

- Criando a tabela **pk_names** para armazenar:
    - name_id (id principal de registro dos pokemons)
    - ref_num (número de referência comumente utilizado nos jogos)
    - name (nome padrão de cada pokemon)
    - type_1 (tipo primário obrigatório para cada pokemon)
    - type_2 (tipo secundário presente em alguns pokemons)
    - gen (referência a qual geração cada pokemon pertence e se é lendário ou não)

    *As colunas type_1 e type_2 serão vinculadas a tabela pk_types e a coluna gen a tabela pk_gen*

In [12]:
pk_names_sql = '''CREATE TABLE IF NOT EXISTS pokedata.public.pk_names (
    name_id                             serial PRIMARY KEY
    ,ref_num                            SMALLINT NOT NULL
    ,name                               VARCHAR(30) UNIQUE NOT NULL
    ,type_1                             INT NOT NULL
    ,type_2                             INT NULL
    ,gen                                INT NOT NULL
    ,FOREIGN KEY (type_1)
        REFERENCES pk_types (type_id)
    ,FOREIGN KEY (type_2)
        REFERENCES pk_types (type_id)
    ,FOREIGN KEY (gen)
        REFERENCES pk_gen (gen_id)
)'''

create_db(pk_names_sql)

- Criando a tabela **pk_status** para armazenar:
    - status_id (usado para vincular cada status a seu respectivo pokemon)
    - attack (usado para armazenar pontuação de ataque comum)
    - defense (usado para armazenar pontuação de defesa comum)
    - sp_atk (usado para armazenar pontuação de ataque especial)
    - sp_def (usado para armazenar pontuação de defesa especial)
    - total (somatório das pontuações de ataque e defesa)

*A coluna status_id será vinculada a seu respectivo pokemon na coluna name_id na tabela pk_names*

In [7]:
pk_status_sql = '''CREATE TABLE IF NOT EXISTS pokedata.public.pk_status (
    status_id                        INT NOT NULL PRIMARY KEY
    ,attack                          SMALLINT DEFAULT 0
    ,defense                         SMALLINT DEFAULT 0
    ,sp_atk                          SMALLINT DEFAULT 0
    ,sp_def                          SMALLINT DEFAULT 0
    ,FOREIGN KEY (status_id)
        REFERENCES pk_names (name_id)
)'''

create_db(pk_status_sql)